In [65]:
import estnltk
from estnltk import Text
import os
import sys
import nltk
from nltk.tokenize.regexp import WhitespaceTokenizer
import html
import re

In [66]:
# 1.1 Korpuse töötlus
def get_tagged_words(sisend, nr):
    margendiga_laused = []
    with open(sisend) as f:
        read = f.readlines()
        uuedRead = []
        for rida in read:
            if (rida.strip() not in ["<p>", "</p>", "<s>"] and len(rida.strip()) != 0):
                uuedRead.append(rida.strip().lower())
        li = []
        for rida in uuedRead:
            if rida.strip() == "</s>":
                if len(li) != 0:
                    margendiga_laused.append(li)
                    li = []
            else:
                rida = html.unescape(rida)
                sona = rida.split(None, 1)[0]
                if nr == 1:
                    margend = re.search(r"//.*_(.*)_", rida).group(1).upper()
                    li.append((sona, margend))
                if nr == 2:
                    li.append(sona)
    return margendiga_laused

def loopFiles(path):
    path = os.getcwd() + '/' + path
    folder = os.listdir(path)
    tagged = []
    for file in folder:
        tagged.extend(get_tagged_words(path + '/' + file, 1))
    return tagged


tagged_texts = loopFiles('train') # loen sisse treeninghulga
test_texts = loopFiles('test') # loen sisse teshulga andmed'

In [67]:
# 1. 2
# kõigepealt teen erinevad märgendajad
train_sents = tagged_texts
default_tagger = nltk.DefaultTagger("S")

In [69]:
# prindin välja erinevate märgendajate evalutaionid
test_sents = test_texts
result = bigram_tagger_backoff.evaluate(test_sents)
print ('{0:.4f} BigramTagger'.format(result))
result = trigram_tagger_backoff.evaluate(test_sents)
print ("{0:.4f} TrigramTagger".format(result))
result = hmm_tagger.evaluate(test_sents)
print ("{0:.4f} HiddenMarkovModelTagger".format(result))

0.9373 BigramTagger
0.9366 TrigramTagger
0.8475 HiddenMarkovModelTagger


In [78]:
result = get_tagged_words(os.getcwd() + '/' + 'ilu_0022.kym', 2)    

In [79]:
bi_tagged = bigram_tagger_backoff.tag_sents(result)
tri_tagged = trigram_tagger_backoff.tag_sents(result)
hmm_tagged = hmm_tagger.tag_sents(result)

In [82]:
df = pd.DataFrame()
df['word'] = [w for s in result for w in s]
df['bi_tag'] = [w[1] for s in bi_tagged for w in s]
df['tri_tag'] = [w[1] for s in tri_tagged for w in s]
df['hmm_tag'] = [w[1] for s in hmm_tagged for w in s]

In [83]:
df.to_csv("Ossip_Villem-Oskar_4.csv", header=False)

In [84]:
# if any differences, filter these to get an overview
unequal = df[(df['bi_tag'] != df['tri_tag']) | (df['bi_tag'] != df['hmm_tag'])]
print("Models predictions contradicted on ", len(unequal), " occasions")

Models predictions contradicted on  304  occasions


In [87]:
def evaluate_estnltk(test_tagged):
    correct = 0
    all_cnt = sum(len(s) for s in test_tagged)
    for s in test_tagged:
        words = [w[0] for w in s]
        tags = [w[1] for w in s]
        lause = " ".join(words)
        lause = Text(lause)
        estnltk_tags = lause.postags
        for i in range(len(tags)):
            if len(estnltk_tags) < i and tags[i] == estnltk_tags[i]:
                correct += 1
    return correct / all_cnt
print(round(evaluate_estnltk(test_sents), 10), "EstNLTK1.4")

0.0 EstNLTK1.4
